# Geo QA

In [3]:
import os
import pandas as pd
from dataclasses import dataclass
from dotenv import load_dotenv

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

In [4]:
load_dotenv()

True

In [16]:
urls = [
    "https://thesaurus.geolba.ac.at/?uri=http://resource.geolba.ac.at/minres/28&lang=de",
    "https://thesaurus.geolba.ac.at/?uri=http://resource.geolba.ac.at/minres/83&lang=de",
    "https://thesaurus.geolba.ac.at/?uri=http://resource.geolba.ac.at/minres/78&lang=de"
]

In [20]:
"""
Overriding the load() function of SeleniumURLLoader
"""
from langchain.document_loaders import SeleniumURLLoader

import logging
logger = logging.getLogger(SeleniumURLLoader.__name__)

from typing import TYPE_CHECKING, List, Literal, Optional, Union
from langchain.docstore.document import Document

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class SeleniumURLLoaderCustom(SeleniumURLLoader):
    def load(self) -> List[Document]:
        """Load the specified URLs using Selenium and create Document instances.

        Returns:
            List[Document]: A list of Document instances with loaded content.
        """
        from unstructured.partition.html import partition_html

        docs: List[Document] = list()
        driver = self._get_driver()

        for url in self.urls:
            try:
                driver.get(url)
                element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".abstract"))
                    # EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".abstract"))
                )
                text = element.text
                # elements = partition_html(text=page_content)
                # text = "\n\n".join([str(el) for el in elements])
                metadata = {"source": url}
                docs.append(Document(page_content=text, metadata=metadata))
            except Exception as e:
                if self.continue_on_failure:
                    logger.error(f"Error fetching or processing {url}, exception: {e}")
                else:
                    raise e

        driver.quit()
        return docs

In [21]:
loader = SeleniumURLLoaderCustom(urls=urls, browser='firefox')

docs = loader.load()
docs

[Document(page_content='Vor allem im Verwitterungsbereich von Braunkohlenvorkommen aus den begleitenden Sulfidkomponenten (Pyrit) hervorgegangenes sulfatisches Mineral, welches insbesondere in der Vergangenheit lokal auch zur Gewinnung von „Alaun“ herangezogen wurde. Ein Gewinnungsort war der Tiefbau Eglsee bei Krems . Alunit wird der Gruppe der Industrieminerale zugeordnet. (Weber & Weiss, 1983)', metadata={'source': 'https://thesaurus.geolba.ac.at/?uri=http://resource.geolba.ac.at/minres/28&lang=de'}),
 Document(page_content='Als Anhydrit wird die wasserfreie Art des Calciumsulfates (CaSO4) verstanden. Durch Aufnahme von Wasser bildet sich Gips (CaSO4·2H2O). Anhydrit wird der Gruppe der Industrieminerale zugeordnet.Anhydrit ist ein wichtiger Rohstoff zur Herstellung von Fließestrich. Bei Wasseraufnahme kann Anhydrit sein Volumen um bis zu 60% vergrößern (Schwellen), weshalb Anhydritvorkommen oft baugeologische Schwierigkeiten nach sich ziehen.Anhydrit tritt mit Gips und anderen Evapo

In [22]:
embedding = OpenAIEmbeddings()

In [23]:
# we already have a persisted DB dir, only need to recompute this on document structure changes 
persist_directory = "chroma/openai-emb"
vectordb = Chroma.from_documents(documents=docs, embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = None

In [27]:
persist_directory = "chroma/openai-emb" # we can load the persisted database from disk, and use it as normal


vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

retriever = vectordb.as_retriever(search_kwargs={"k": 8})

llm = OpenAI(model_name="text-davinci-003", 
             temperature=0.9, 
             max_tokens=1000)

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff", 
                                 retriever=retriever,
                                 verbose=True)

In [28]:
prompt = 'Hallo!'

qa.run(prompt)



> Entering new  chain...


Number of requested results 8 is greater than number of elements in index 3, updating n_results = 3



> Finished chain.


' Hallo! Wie kann ich Ihnen helfen?'